# RTL Wiki. Preprocessing

Example of a text data preprocessing before topic modeling (applied to RTL-Wiki dataset).

The dataset was introduced in [1] and later recreated in [2]. You can download it in from http://139.18.2.164/mroeder/palmetto/datasets/rtl-wiki.tar.gz

References:

* [1] "Reading Tea Leaves: How Humans Interpret Topic Models" (NIPS 2009)  
* [2] "Exploring the Space of Topic Coherence Measures" (WSDM 2015)
* Related notebook (with preprocessing for other dataset): [20NG-Preprocessing.ipynb](./20NG-Preprocessing.ipynb)

# Contents<a id="contents"></a>

* [Data loading](#data-loading)
* [Data filtering](#data-filtering)
* [Text preprocessing (lemmatization, stopwords filtering, N-grams extraction)](#text-preprocessing)
* [Vowpal Wabbit](#vowpal-wabbit)

In [ ]:
import glob
import nltk
import pandas as pd
import re
import string

from bs4 import BeautifulSoup
from collections import Counter
from mwparserfromhell import mwparserfromhell
from nltk.collocations import (
    BigramAssocMeasures,
    BigramCollocationFinder,
)
from nltk.corpus import (
    stopwords,
    wordnet,
)
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm_notebook as tqdm

## Data loading<a id="data-loading"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

In [1]:
# download corpus and unpack it:

! wget http://139.18.2.164/mroeder/palmetto/datasets/rtl-wiki.tar.gz -O rtl-wiki.tar.gz
! tar xzf rtl-wiki.tar.gz

--2020-01-25 23:56:21--  http://139.18.2.164/mroeder/palmetto/datasets/rtl-wiki.tar.gz
Connecting to 139.18.2.164:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 176728881 (169M) [application/octet-stream]
Saving to: ‘rtl-wiki.tar.gz’

rtl-wiki.tar.gz     100%[===================>] 168,54M  12,5MB/s    in 12s     

2020-01-25 23:56:34 (13,6 MB/s) - ‘rtl-wiki.tar.gz’ saved [176728881/176728881]



The corpus is a sample of 10000 articles from English Wikipedia in a MediaWiki markup format.

Hence, we need to strip specific wiki formatting. We advise using a `mwparserfromhell` fork optimized to deal with the English Wikipedia.

```bash
git clone --branch images_and_interwiki https://github.com/bt2901/mwparserfromhell.git
```

In [2]:
! git clone --branch images_and_interwiki https://github.com/bt2901/mwparserfromhell.git

Cloning into 'mwparserfromhell'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 5756 (delta 15), reused 19 (delta 9), pack-reused 5722
Receiving objects: 100% (5756/5756), 1.75 MiB | 2.78 MiB/s, done.
Resolving deltas: 100% (3987/3987), done.


## Data filtering<a id="data-filtering"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

The Wikipedia dataset is too heterogenous. Building a good topic model here requires a lot of topics or a lot of documents.

To make collection more focused, we will filter out everything which isn't about people. We will use the following criteria to distinguish people and not-people:




In [1]:
# all infoboxes related to persons, according to  https://en.wikipedia.org/wiki/Wikipedia:List_of_infoboxes
person_infoboxes = {'infobox magic: the gathering player', 'infobox architect', 'infobox mountaineer', 'infobox scientist', 'infobox chess biography', 'infobox racing driver', 'infobox saint', 'infobox snooker player', 'infobox figure skater', 'infobox theological work', 'infobox gaelic athletic association player', 'infobox professional wrestler', 'infobox noble', 'infobox pelotari', 'infobox native american leader', 'infobox pretender', 'infobox amateur wrestler', 'infobox college football player', 'infobox buddha', 'infobox cfl biography', 'infobox playboy playmate', 'infobox cyclist', 'infobox martial artist', 'infobox motorcycle rider', 'infobox motocross rider', 'infobox bandy biography', 'infobox video game player', 'infobox dancer', 'infobox nahua officeholder', 'infobox criminal', 'infobox squash player', 'infobox go player', 'infobox bullfighting career', 'infobox engineering career', 'infobox pirate', 'infobox latter day saint biography', 'infobox sumo wrestler', 'infobox youtube personality', 'infobox national hockey league coach', 'infobox rebbe', 'infobox football official', 'infobox aviator', 'infobox pharaoh', 'infobox classical composer', 'infobox fbi ten most wanted', 'infobox chef', 'infobox engineer', 'infobox nascar driver', 'infobox medical person', 'infobox jewish leader', 'infobox horseracing personality', 'infobox poker player', 'infobox economist', 'infobox peer', 'infobox war on terror detainee', 'infobox philosopher', 'infobox professional bowler', 'infobox champ car driver', 'infobox golfer', 'infobox le mans driver', 'infobox alpine ski racer', 'infobox boxer (amateur)', 'infobox bodybuilder', 'infobox college coach', 'infobox speedway rider', 'infobox skier', 'infobox medical details', 'infobox field hockey player', 'infobox badminton player', 'infobox sports announcer details', 'infobox academic', 'infobox f1 driver', 'infobox ncaa athlete', 'infobox biathlete', 'infobox comics creator', 'infobox rugby league biography', 'infobox fencer', 'infobox theologian', 'infobox religious biography', 'infobox egyptian dignitary', 'infobox curler', 'infobox racing driver series section', 'infobox afl biography', 'infobox speed skater', 'infobox climber', 'infobox rugby biography', 'infobox clergy', 'infobox equestrian', 'infobox member of the knesset', 'infobox pageant titleholder', 'infobox lacrosse player', 'infobox tennis biography', 'infobox gymnast', 'infobox sport wrestler', 'infobox sports announcer', 'infobox surfer', 'infobox darts player', 'infobox christian leader', 'infobox presenter', 'infobox gunpowder plotter', 'infobox table tennis player', 'infobox sailor', 'infobox astronaut', 'infobox handball biography', 'infobox volleyball biography', 'infobox spy', 'infobox wrc driver', 'infobox police officer', 'infobox swimmer', 'infobox netball biography', 'infobox model', 'infobox comedian', 'infobox boxer'}

# is page included in a category with demography information?
demography_re = re.compile("([0-9]+ (deaths|births))|(living people)")

In [2]:
dir_name = "persons"

In [3]:
! mkdir $dir_name

In [4]:
for filename in tqdm(glob.glob("documents/*.html")):
    doc_id = filename.partition("/")[-1]
    doc_id = doc_id.rpartition(".")[0] + ".txt"
    is_about_person = False

    with open(filename, "r") as f:
        soup = BeautifulSoup("".join(f.readlines()))
        text = soup.findAll('textarea', id="wpTextbox1")[0].contents[0]
        text = text.replace("&amp;", "&").replace('&lt;', '<').replace('&gt;', '>')
        wikicode = mwparserfromhell.parse(text)

        if dir_name == "persons":
            for node in wikicode.nodes:
                entry_type = str(type(node))

                if "Wikilink" in entry_type:
                    special_link_name, _, cat_name = node.title.lower().strip().partition(":")
                    if special_link_name == "category":
                        if demography_re.match(cat_name):
                            is_about_person = True

                if "Template" in entry_type:
                    name = str(node.name).lower().strip()
                    if name in person_infoboxes:
                        is_about_person = True

            should_be_saved = is_about_person
        else:
            should_be_saved = True

        if should_be_saved:
            with open(f"{dir_name}/{doc_id}", "w") as f2:
                stripped_text = wikicode.strip_code()
                f2.write(stripped_text)

Now we have a folder `persons` which contains 1201 document. Let's take a look at the one of them:

In [5]:
! head $dir_name/Eusebius.txt

Eusebius of Caesarea
Eusebius of Caesarea (c. 263 – c. 339) (often called Eusebius Pamphili, "Eusebius [the friend] of Pamphilus") became the bishop of Caesarea Palaestina, the capital of Iudaea province, c 314. He is often referred to as the Father of Church History because of his work in recording the history of the early Christian church, especially Chronicle and Ecclesiastical History Wetterau, Bruce. World history. New York: Henry Holt and company. 1994. . 

Biography

His  date and place of birth are unknown and little is known of his youth, however it is estimated that he was born in 265 Encyclopedia of the Early Church, Published in 1992, English Version, page 299 . He became acquainted with the presbyter Dorotheus in Antioch and probably received exegetical instruction from him. In 296 he was in Palestine and saw Constantine who visited the country with Diocletian. He was in Caesarea when Agapius was bishop and became friendly with Pamphilus of Caesarea, with whom he seems to 

## Text preprocessing (lemmatization, stopwords filtering, N-grams extraction)<a id="text-preprocessing"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

We need to lemmatize texts, remove stopwords and extract informative ngramms.

There's no one "correct" way to do it, but the reasonable baseline is using well-known `nltk` library.

In [7]:
nltk.data.path.append('/home/evgenyegorov/nltk_data/')

In [8]:
files = glob.glob(dir_name + '/*.txt')

In [9]:
data = []

for path in files:
    entry = {}
    entry['id'] = path.split('/')[-1].rpartition(".")[0]

    with open(path, 'r') as f:
        entry['raw_text'] = " ".join(line.strip() for line in f.readlines())

    data.append(entry)

wiki_texts = pd.DataFrame(data)

In [11]:
tokenized_text = []

for text in tqdm(wiki_texts['raw_text'].values):
    tokens = nltk.wordpunct_tokenize(text.lower())
    tokenized_text.append(nltk.pos_tag(tokens))

wiki_texts['tokenized'] = tokenized_text

100%|██████████| 1201/1201 [01:37<00:00, 12.29it/s]


In [12]:
def nltk2wn_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return ''

In [13]:
stop = set(stopwords.words('english'))

lemmatized_text = []
wnl = WordNetLemmatizer()

for text in wiki_texts['tokenized'].values:
    lemmatized = [wnl.lemmatize(word,nltk2wn_tag(pos))
                  if nltk2wn_tag(pos) != ''
                  else wnl.lemmatize(word)
                  for word, pos in text ]
    lemmatized = [word for word in lemmatized 
                  if word not in stop and word.isalpha()]
    lemmatized_text.append(lemmatized)

wiki_texts['lemmatized'] = lemmatized_text

Ngrams are a powerful feature, and BigARTM is able to take advantage of it (the technical term is 'multimodal topic modeling': our topic model could model a lot of different features linked to a specific document, not just words).

In [14]:
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_documents(wiki_texts['lemmatized'])

finder.apply_freq_filter(5)
set_dict = set(finder.nbest(bigram_measures.pmi,32100)[100:])
documents = wiki_texts['lemmatized']
bigrams = []

for doc in documents:
    entry = ['_'.join([word_first, word_second])
             for word_first, word_second in zip(doc[:-1],doc[1:])
             if (word_first, word_second) in set_dict]
    bigrams.append(entry)

wiki_texts['bigram'] = bigrams

## Vowpal Wabbit<a id="vowpal-wabbit"></a>

<div style="text-align: right">Back to <a href=#contents>Contents</a></div>

* More info about different text formats (including Vowpal Wabbit): [bigartm.formats.html](https://bigartm.readthedocs.io/en/v0.8.1/formats.html)
* Example of a toy collection in Vowpal Wabbit format: [sample_vw.txt](https://github.com/machine-intelligence-laboratory/OptimalNumberOfTopics/blob/master/sample/vw.txt)

In [15]:
def vowpalize_sequence(sequence):
    word_2_frequency = Counter(sequence)

    del word_2_frequency['']

    vw_string = ''

    for word in word_2_frequency:
        vw_string += word + ":" + str(word_2_frequency[word]) + ' '

    return vw_string

In [16]:
vw_text = []

for index, data in wiki_texts.iterrows():
    vw_string = ''    
    doc_id = data.id
    lemmatized = '@lemmatized ' + vowpalize_sequence(data.lemmatized)
    bigram = '@bigram ' + vowpalize_sequence(data.bigram)
    vw_string = ' |'.join([doc_id, lemmatized, bigram])
    vw_text.append(vw_string)

wiki_texts['vw_text'] = vw_text

Vowpal Wabbit ("wv") is a text format which is a good fit for multimodal topic modeling. Here, we elected to store dataset in a Bag-of-Words format (for performance reasons), but VW could store everything as a sequence of words as well.

It looks like this:

In [17]:
wiki_texts['vw_text'].head().values[0]

'İsmet_İnönü |@lemmatized mustafa:2 smet:7 nönü:18 september:1 december:2 turkish:25 army:8 general:5 tsk:1 genel:1 kurmay:1 baskanlari:1 prime:5 minister:5 second:4 president:6 republic:4 turkey:19 widely:1 refer:1 milli:2 şef:2 national:5 chief:5 title:2 bestow:1 upon:1 elect:2 family:3 early:5 life:2 bear:2 zmir:2 originally:1 malatya:3 mixed:1 kurdish:1 heritage:1 young:2 turk:2 child:1 borderland:1 erik:1 jan:1 zürcher:1 universiteit:1 leiden:1 ismet:2 inonu:1 making:1 statesman:1 metin:1 heper:1 brill:1 academic:1 publisher:1 father:2 hacı:1 reşid:1 bey:1 member:2 ottoman:5 bureaucracy:1 examine:1 magistrate:1 mother:1 cevriye:1 hanım:1 daughter:1 russo:1 war:19 refugee:1 bulgaria:1 due:3 assignment:3 move:1 one:1 city:2 another:2 thus:1 complete:1 primary:1 education:1 siva:1 half:1 know:3 pasha:3 change:1 name:2 atatürk:8 decree:1 countryman:1 surname:2 decide:1 take:2 central:1 anatolian:1 town:1 command:1 force:3 great:2 battle:6 first:8 victory:3 play:1 important:1 role:1 in

In [18]:
wiki_texts[['id','raw_text', 'vw_text']].to_csv('./wiki_data.csv')